In [1]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
import seaborn as sns

# Carregue seus dados
df_train = pd.read_csv("train.csv")
df_test = pd.read_csv("test.csv")

# Defina suas variáveis dependentes e independentes
y_train = df_train["smoking"]
X_train = df_train.drop("smoking", axis=1)
X_test = df_test

# Divida seus dados em conjuntos de treinamento e validação
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.1, random_state=5)

# Crie um modelo Random Forest
rf = RandomForestClassifier(random_state=5, class_weight='balanced', n_jobs=-1)

# Ajuste o modelo aos dados de treinamento
rf.fit(X_train, y_train)

# Avalie o desempenho no conjunto de validação
score_val = accuracy_score(y_val, rf.predict(X_val))
print("Pontuação no conjunto de validação:", score_val)

# Defina os hiperparâmetros que você deseja otimizar
parameters = {
    'n_estimators': [100,200,300],  # Número de árvores na floresta
    'max_depth': [2,4,6],  # Profundidade máxima das árvores
}

# Realize uma pesquisa em grade para encontrar os melhores hiperparâmetros
grid_search = GridSearchCV(rf, parameters, n_jobs=-1, cv=StratifiedKFold(n_splits=5, shuffle=True, random_state=5))
grid_search.fit(X_train, y_train)

# Imprima o melhor modelo encontrado durante a pesquisa em grade
print("Melhor modelo:", grid_search.best_estimator_)

# Imprima a pontuação do melhor modelo
score_best = grid_search.best_score_
print("Pontuação do melhor modelo:", score_best)

# Faça previsões de probabilidade no conjunto de teste
y_test_probabilities = rf.predict_proba(X_test)[:, 1]

# Crie um DataFrame de submissão com as probabilidades de previsão
df_submission = pd.DataFrame({"id": df_test["id"], "smoking": y_test_probabilities})

# Salve o DataFrame de submissão em um arquivo CSV
df_submission.to_csv("sample_submission_rf_probabilities.csv", index=False)



Pontuação no conjunto de validação: 0.7742684917744569
Melhor modelo: RandomForestClassifier(class_weight='balanced', max_depth=6, n_jobs=-1,
                       random_state=5)
Pontuação do melhor modelo: 0.7517825995953394
